<a href="https://colab.research.google.com/github/smargetic/Natural_Language_Processing/blob/main/Machine_Translation/Neural_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nbconvert

In [2]:
#data storage - not sure if important
import pandas as pd
import numpy as np #can take away later

#make sure to show all values in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

#pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F

#visualize
from IPython.display import display

#help import other functions
import os

#mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Disable scientific notation
torch.set_printoptions(sci_mode=False)

Mounted at /content/drive


In [3]:
##to obtain functions from another .ipynb file

#go to current directory
os.chdir('/content/drive/My Drive/Coding Projects/Machine Translation')

#List the files in the current directory
print('\nFiles currently present:')
print(os.listdir())

#Convert the notebook to a Python script
!jupyter nbconvert --to script intake_preprocess_utils.ipynb
os.rename('intake_preprocess_utils.txt', 'intake_preprocess_utils.py')

print('\n\nNew files in current directory:')
print(os.listdir())
print('\n\n')

#import functions
from intake_preprocess_utils import get_file_and_disp, get_sample, dump_sample, data_preprocessing


Files currently present:
['sentences_base.csv', 'ita_sentences_CC0.tsv', 'eng_sentences_detailed.tsv', 'eng_sentences.tsv', 'ita_sentences_detailed.tsv', 'eng_sentences_CC0.tsv', 'ita_sentences.tsv', 'Sentence pairs in English-Italian - 2024-07-26.tsv', 'Helper_Functions.ipynb', 'Model_Train_Predict.ipynb', '__pycache__', 'Data_Intake_and_Preprocessing.ipynb', 'intake_preprocess_utils.ipynb', 'intake_preprocess_utils.py', 'Neural_Net.ipynb']
[NbConvertApp] Converting notebook intake_preprocess_utils.ipynb to script
[NbConvertApp] Writing 11518 bytes to intake_preprocess_utils.txt


New files in current directory:
['sentences_base.csv', 'ita_sentences_CC0.tsv', 'eng_sentences_detailed.tsv', 'eng_sentences.tsv', 'ita_sentences_detailed.tsv', 'eng_sentences_CC0.tsv', 'ita_sentences.tsv', 'Sentence pairs in English-Italian - 2024-07-26.tsv', 'Helper_Functions.ipynb', 'Model_Train_Predict.ipynb', '__pycache__', 'Data_Intake_and_Preprocessing.ipynb', 'intake_preprocess_utils.ipynb', 'intake

In [4]:
# #english to italian translation - just to test
# df_eng_it = get_file_and_disp(fileName="Sentence pairs in English-Italian - 2024-07-26.tsv.zip" ,sep='\t', stringName="English to Italian")

In [5]:
#test data set
df = pd.DataFrame({
                  0:[1,2,3],
                  1:["hello world", "how are you", "where are we going"],
                  2:[1,2,3],
                  3: ["ciao mondo", "come stai", "dove stiamo andando"]})

display(df)




,0,1,2,3
0,1,hello world,1,ciao mondo
1,2,how are you,2,come stai
2,3,where are we going,3,dove stiamo andando


In [6]:
#go through preprocessing
data_dict= data_preprocessing(df.copy(), pytorchB=True, tensorflowB=False, store=False, sample=False, download=False, sing=False)
for key in data_dict.keys():
  print('\n'+key+':')
  display(data_dict[key])

Nulls in English: 0
Nulls in Italian: 0

English/Italian Translations:
	English vocabulary size: 11
	Italian vocabulary size: 10

Done with Pytorch.

English-Italian Dataframe:


,eng_id,eng_sentence,it_id,it_sentence,eng_tokens,it_tokens,eng_tokens_enc,it_tokens_enc,eng_tokens_enc_p,it_tokens_enc_p
0,1,hello world,1,ciao mondo,"[<sos>, hello, world, <eos>]","[<sos>, ciao, mondo, <eos>]","[1, 5, 9, 2]","[1, 4, 7, 2]","[tensor(1), tensor(5), tensor(9), tensor(2), tensor(0), tensor(0)]","[tensor(1), tensor(4), tensor(7), tensor(2), tensor(0)]"
1,2,how are you,2,come stai,"[<sos>, how, are, you, <eos>]","[<sos>, come, stai, <eos>]","[1, 6, 3, 10, 2]","[1, 5, 8, 2]","[tensor(1), tensor(6), tensor(3), tensor(10), tensor(2), tensor(0)]","[tensor(1), tensor(5), tensor(8), tensor(2), tensor(0)]"
2,3,where are we going,3,dove stiamo andando,"[<sos>, where, are, we, going, <eos>]","[<sos>, dove, stiamo, andando, <eos>]","[1, 8, 3, 7, 4, 2]","[1, 6, 9, 3, 2]","[tensor(1), tensor(8), tensor(3), tensor(7), tensor(4), tensor(2)]","[tensor(1), tensor(6), tensor(9), tensor(3), tensor(2)]"



English-Italian SING Dataframe:


None


English-Italian Pytorch Dataloader:


...

{'<sos>': 1,
 '<eos>': 2,
 'andando': 3,
 'ciao': 4,
 'come': 5,
 'dove': 6,
 'mondo': 7,
 'stai': 8,
 'stiamo': 9,
 '<pad>': 0}


English Vocabulary SING:


None


Italian Vocabulary SING:


None

In [7]:
#forward input: (batch size (aka. # sentences), # words in sentence (including padding))
#forward output: (batch size (aka. # sentences), # words in sentence (including padding), embedding dim)
class Embedding_Node(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(Embedding_Node, self).__init__()

    self.W_e = nn.Parameter(torch.randn(vocab_size, embedding_dim))

  def forward(self, x):
    return self.W_e[x]

In [8]:
### Seperates data into batches --> NEED TO GO OVER

for input_batch, target_batch in data_dict['English-Italian Pytorch Dataloader']:
  print('\nbatch:')
  display(input_batch)
  display(target_batch)

print('\n\nbatch:')
display(input_batch)
display(target_batch)

#get input and output
input, output = input_batch, target_batch


batch:


tensor([[ 1,  6,  3, 10,  2,  0],
        [ 1,  8,  3,  7,  4,  2],
        [ 1,  5,  9,  2,  0,  0]])

tensor([[1, 5, 8, 2, 0],
        [1, 6, 9, 3, 2],
        [1, 4, 7, 2, 0]])



batch:


tensor([[ 1,  6,  3, 10,  2,  0],
        [ 1,  8,  3,  7,  4,  2],
        [ 1,  5,  9,  2,  0,  0]])

tensor([[1, 5, 8, 2, 0],
        [1, 6, 9, 3, 2],
        [1, 4, 7, 2, 0]])

In [9]:
class Attention_Node(nn.Module):
  def __init__(self,hidden_dim):
    self.hidden_dim = hidden_dim

    #key, query, value
    self.W_key = nn.Linear(hidden_dim, hidden_dim)
    self.W_query = nn.Linear(hidden_dim, hidden_dim)
    self.W_value = nn.Linear(hidden_dim, hidden_dim)

  def forward(self, key_hidden, query_hidden, masked=False):
    #key, query, value
    key = self.W_key(key_hidden)
    query = self.W_query(query_hidden)
    value = self.W_value(key_hidden)

    #get attention scores
    attention_score = torch.matmul(query.T, key)

    #masked - make upper triangle of values negative inf
    if(masked):
      mask = torch.triu(torch.ones_like(attention_score), diagonal=0).bool()
      attention_score[~mask] = float('-inf')

    #softmax
    attention_dist = F.softmax(attention_score, dim=0)

    #attention output - weighted sum
    attention_output = torch.dot(attention_dist, value)

    return attention_output

In [10]:
# att_node = Attention_Node(embedding_dim)

In [11]:
class RNN_Node(nn.Module):
  def __init__(self, hidden_dim, input_dim):
    super(RNN_Node, self).__init__()

    #all parameters that go into the rnn node
    self.layer_weight = nn.Parameter(torch.randn(hidden_dim, input_dim))
    self.hidden_weight = nn.Parameter(torch.randn(hidden_dim, hidden_dim))
    self.bias = nn.Parameter(torch.zeros(hidden_dim))

    #store dimensions
    self.hidden_dim = hidden_dim
    self.input_dim = input_dim

  #forward pass is how its connected to the next rnn node (regardless of layer or column)
  def forward(self, input, prev_hidden):
    #broadcast bias
    bias = self.bias.expand(input.shape[0], self.hidden_dim)

    #get hidden node
    hidden_node = torch.sigmoid(torch.matmul(self.hidden_weight, prev_hidden.T).T+
                                     torch.matmul(self.layer_weight, input.T).T+
                                     bias) #could also do tanh instead of sigmoid

    return hidden_node


In [12]:
class LSTM_Node(nn.Module):
  def __init__(self, hidden_dim, embed_dim):
    super(LSTM_Node, self).__init__()

    #forget weight / prev layer forget weight / forget bias
    self.W_f = nn.Parameter(torch.randn(hidden_dim, hidden_dim))
    self.U_f = nn.Parameter(torch.randn(hidden_dim, embed_dim))
    self.b_f = nn.Parameter(torch.zeros(hidden_dim))

    #input weight / prev layer input weight / input bias
    self.W_i = nn.Parameter(torch.randn(hidden_dim, hidden_dim))
    self.U_i = nn.Parameter(torch.randn(hidden_dim, embed_dim))
    self.b_i = nn.Parameter(torch.zeros(hidden_dim))

    #output weight / prev layer output weight / output bis
    self.W_o = nn.Parameter(torch.randn(hidden_dim, hidden_dim))
    self.U_o = nn.Parameter(torch.randn(hidden_dim, embed_dim))
    self.b_o = nn.Parameter(torch.zeros(hidden_dim))

    #cell weight / prev layer cell weight / cell bias
    self.W_c = nn.Parameter(torch.randn(hidden_dim, hidden_dim))
    self.U_c = nn.Parameter(torch.randn(hidden_dim, embed_dim))
    self.b_c = nn.Parameter(torch.zeros(hidden_dim))

    #store dimensions
    self.hidden_dim = hidden_dim
    self.embed_dim = embed_dim


  #forward pass is how its connected to the next lstm node (regardless of layer or column)
  def forward(self, input, prev_hidden, prev_cell):


    #broadcast bias
    b_f = self.b_f.expand(input.shape[0], self.hidden_dim)
    b_i = self.b_i.expand(input.shape[0], self.hidden_dim)
    b_o = self.b_o.expand(input.shape[0], self.hidden_dim)
    b_c = self.b_c.expand(input.shape[0], self.hidden_dim)


    #forget
    f_t = torch.sigmoid(torch.matmul(self.W_f, prev_hidden.T).T+
                                     torch.matmul(self.U_f, input.T).T+
                                     b_f)
    #input
    i_t = torch.sigmoid(torch.matmul(self.W_i, prev_hidden.T).T+
                                     torch.matmul(self.U_i, input.T).T+
                                     b_i)
    #output
    o_t = torch.sigmoid(torch.matmul(self.W_o, prev_hidden.T).T+
                                     torch.matmul(self.U_o, input.T).T+
                                     b_o)

    #new cell data
    nc_t = torch.tanh(torch.matmul(self.W_c, prev_hidden.T).T+
                                     torch.matmul(self.U_c, input.T).T+
                                     b_c)

    #cell
    c_t = f_t*prev_cell + i_t*nc_t

    #hidden
    h_t = o_t*torch.tanh(c_t)

    return h_t, c_t


In [13]:
def add_norm(prev_x, new_x):
  x = torch.cat(prev_x, new_x, dim=0)
  x = nn.LayerNorm(x)
  return x

In [14]:
class Transformer_Node(nn.Module):
  def __init__(self, hidden_dim, embed_dim, ff_dim, decoder=False):
    super(Transformer_Node, self).__init__()

    ##layers##
    #attention
    self.attention = Attention_Node(hidden_dim)
    self.decoder = decoder
    if(decoder):
      self.decoder_encoder_att = Attention_Node(hidden_dim)

    #feed forwards
    self.linear1 = nn.Linear(hidden_dim, ff_dim)
    self.linear2 = nn.Linear(ff_dim, hidden_dim)


  #how its connected to the next transformer node
  def forward(self, inputs, encoder_hidden=None):
    #multihead attention (with mask option) ?
    if(self.decoder): # if decoder, only look at values before
      attention_out = self.attention.forward(inputs, inputs, masked=True)
    else:
      attention_out = self.attention.forward(inputs, inputs)

    #add and norm
    x = add_norm(inputs, attention_out)

    #encoder decoder attention
    if(self.decoder):
      x_prev = x
      self.decoder_encoder_att.forward(encoder_hidden, x)
      x = add_norm(x_prev, x)

    x_prev = x
    #feed forward
    x = self.linear1(x)
    x = self.linear2(x)

    #activation function
    x = nn.relu(x)

    #add and norm
    x = add_norm(x_prev, x)

    return x


In [15]:
# class Transformer_Encoder(nn.Module):
#   def __init__(self, hidden_dim, n_layers, vocab_size):
#     super(Transformer_Encoder, self).__init__()

#     #embedding
#     self.embedding_node = Embedding_Node(vocab_size, hidden_dim)

#     #layers
#     self.layers = nn.ModuleList([Transformer_Node(hidden_dim) for i in range(0,n_layers)])

#   def forward(self, inputs):
#     pass

In [16]:
# class Transformer_Node(nn.Module):
#   def __init__(self, hidden_dim):
#     super(Transformer_Node, self).__init__()

#   def forward_encoder(self, inputs):
#     pass

#   def forward_decoder(self, inputs):
#     pass

In [17]:
#for this, we basically said that what should be in the __init__ function should be any parameters that could be learned

#ok, so for node elements, we'll say that in the __init__ funciton we should have the parameters whos weight should be learned
#but for nn layers, well have things that define the layers/ hyperparameters

#__init__ layer = anything that needs to be consistently referenced in forward pass (especially parameters that need to be updated)

In [18]:
  # #define initial hidden states at run time - b/c of batch size
  # def init_hidden(self, batch_size):
  #   init_hidden_list = [torch.zeros(batch_size, self.hidden_dim) for _ in range(self.n_layers)]
  #   init_cell_list = None
  #   if(self.node_type=="lstm"):
  #     init_cell_list = [torch.zeros(batch_size, self.hidden_dim) for _ in range(self.n_layers)]
  #   return init_hidden_list, init_cell_list

In [19]:
#define initial hidden states at run time - b/c of batch size
def init_hidden(self, batch_size):
  #get device
  device = next(self.parameters()).device

  #itialize tens based on node type
  init_hidden_tensor = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
  init_cell_tensor = None
  if(self.node_type=="lstm"):
    init_cell_tensor = torch.zeros(self.n_laryers, batch_size, self.hidden_dim).to(device)
  return init_hidden_tensor, init_cell_tensor

In [20]:
# input.shape[1]

In [21]:
class EncoderNN(nn.Module):
  def __init__(self, node_type, n_layers, hidden_dim, embed_dim, vocab_size):
    super(EncoderNN, self).__init__()
    # self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #model parameters
    self.n_layers = n_layers
    self.node_type = node_type
    self.hidden_dim = hidden_dim

    #define embedding node
    self.embedding_node = Embedding_Node(vocab_size, embed_dim)


    #create layer of nodes based on type
    node_class = RNN_Node if node_type == "rnn" else LSTM_Node
    self.nodes = nn.ModuleList(
          [node_class(hidden_dim, embed_dim)] +
          [node_class(hidden_dim, hidden_dim) for _ in range(n_layers - 1)])

  #forward pass
  def forward(self, inputs):

    #get blank nodes
    node_list = self.nodes

    #get shape params
    batch_size = inputs.shape[0]
    seq_size = inputs.shape[1]

    #get initial states
    # hidden_list, cell_list = self.init_hidden(inputs.shape[0])
    hidden_tensor, cell_tensor = init_hidden(self, batch_size)
    last_layer_hidden = torch.zeros(batch_size, seq_size, self.hidden_dim)

    #turn input into embedding
    embed_input = self.embedding_node.forward(inputs)

    # print('this is the embedding inputs:')
    # display(embed_input)

    #store final layer hidden nodes (for attention)
    # final_hidden_layer = []
    # final_hidden_tensor = torch.zeros(batch_size, )


    #iterate over each word in sentence
    for word_i in range(0, seq_size):

      input = embed_input[:,word_i] #input = (batch_size, hidden_dim) ##1 for each word

      for layer in range(0,self.n_layers):
        #if not first layer, then input is previous hidden state
        if layer!=0:
          input = hidden_tensor[layer-1]

        #TRY TO SEE IF CAN MAKE NEATER
        #forward pass -- TRY TO FIX LATER - AFTER TRANSFORMER
        if(self.node_type=="rnn"):
          hidden_tensor[layer] = node_list[layer].forward(input, hidden_tensor[layer]) # (batch, hidden dim)
        elif(self.node_type=='lstm'):
          hidden_tensor[layer], cell_tensor[layer] = node_list[layer].forward(input, hidden_tensor[layer], cell_tensor[layer]) # both of size (batch, hidden dim)

      #store final hidden layer for each word - for attention
      last_layer_hidden[:, word_i, :self.hidden_dim] = hidden_tensor[-1]

    #format outputs
    outputs = (hidden_tensor[-1], None) if self.node_type=="rnn" else (hidden_tensor[-1], cell_tensor[-1]) if self.node_type=='lstm' else None

    return outputs, last_layer_hidden

In [22]:
class BeamNode(nn.Module):
  def __init__(self, pred_vocab_indxs=[], pred_probs=[-np.inf], hidden_tens=None, cell_tens=None, parent_node=None):
    super(BeamNode, self).__init__()

    # #relationship
    # self.parent_node = parent_node

    # #values
    # self.pred_prob = pred_prob
    # self.pred_vocab_indx = pred_vocab_indx

    #running values
    self.pred_probs = pred_probs
    self.pred_vocab_indxs = pred_vocab_indxs

    #recurrent states
    self.hidden_tens = hidden_tens
    self.cell_tens = cell_tens

    #current state
    self.eof_flag = 0

  def add_pred_probs(self, val):
    self.pred_probs.append(val)

  def add_pred_vocab_indx(self, val):
    self.pred_vocab_indxs.append(val)

  #calculate average log prob for comparing results
  def avg_log_prob(self):
    norm_score = np.sum(np.log(self.pred_probs))/len(self.pred_probs)
    return norm_score


In [34]:
class DecoderNN(nn.Module):
  def __init__(self, node_type, n_layers, hidden_dim, embed_dim, vocab_size, attention=False):
    super(DecoderNN, self).__init__()

    #model parameters
    self.n_layers = n_layers
    self.node_type = node_type
    # self.max_length = max_length
    self.hidden_dim = hidden_dim
    self.vocab_size = vocab_size

    #define embedding node
    self.embedding_node = Embedding_Node(vocab_size, embed_dim)

    #create layer of nodes based on type
    node_class = RNN_Node if node_type == "rnn" else LSTM_Node
    self.nodes = nn.ModuleList(
          [node_class(hidden_dim, embed_dim)] +
          [node_class(hidden_dim, hidden_dim) for _ in range(n_layers - 1)])


    # # #store initial hidden states
    # # #self.hidden = torch.randn(n_layers, hidden_dim)
    # # self.cell = torch.randn(n_layers, hidden_dim) #only really applicable for lstm
    # self.prev_hidden = torch.randn(n_layers, hidden_dim)
    # self.prev_cell = torch.randn(n_layers, hidden_dim) #only really applicable for lstm


    #output weights
    self.W_out = nn.Parameter(torch.randn(vocab_size, hidden_dim))

    #attention
    self.attention = attention
    if(attention):
      self.attention_node = Attention_Node(hidden_dim)


  # #define initial hidden states at run time - b/c of batch size
  # def init_hidden(self, batch_size):
  #   init_hidden_list = [torch.zeros(batch_size, hidden_dim) for _ in range(self.n_layers)]
  #   init_cell_list = None
  #   if(self.node_type=="lstm"):
  #     init_cell_list = [torch.zeros(batch_size, hidden_dim) for _ in range(self.n_layers)]
  #   return init_hidden_list, init_cell_list

  #forward pass - considered as going over one word (versus encoder going over entire sentence) - training
  def forward(self, input, hidden_tensor, cell_tensor=None, encoder_flayer=None):

    #initial nodes
    node_list = self.nodes

    #get embedding of input
    input = self.embedding_node.forward(input)[:,0] #only take one word

    #forward pass for all layers (remember - not entire sentence, just 1 word)
    for layer in range(0,self.n_layers):
      #if not first layer, then input is previous hidden state
      if layer!=0:
        input = hidden_tensor[layer-1]

      #forward pass
      if(self.node_type=="rnn"):
        hidden_tensor[layer] = node_list[layer].forward(input, hidden_tensor[layer]) # (batch, hidden dim)
      elif(self.node_type=='lstm'):
        hidden_tensor[layer], cell_tensor[layer] = node_list[layer].forward(input, hidden_tensor[layer], cell_tensor[layer]) # both of size (batch, hidden dim)

      # print('\nLAYER {}:'.format(layer))
      # display(hidden_tensor)


    #COME BACK TO ATTENTION
    #attention
    # if(self.attention):
    #   attention_out = self.attention_node.forward(encoder_flayer, hidden_list[-1])
    #   hidden_list[-1] = torch.cat((hidden_list[-1], attention_out), dim=0)

    #generate final output
    pred_output = torch.matmul(self.W_out, hidden_tensor[-1].T).T #during training - cross entropy fun converts this to prob

    return hidden_tensor, cell_tensor, pred_output

  #to generate predictions - generate one word at a time
  #greedy decoding - uses/ predicts most probable word - singular
  def pred(self, input, hidden_tens, cell_tens, encoder_flayer=None, sos_token=1,pad_token=0,eos_token=2):
    # print('\nIN DECODER PREDICTIONS')

    # #turn to list
    # prev_input_nodes = prev_input_nodes.tolist()

    # print('\nHIDDEN STATE IN ONE NODE')
    # display(prev_input_nodes[0].hidden_tens)
    # print('shape: {}'.format(prev_input_nodes[0].hidden_tens.shape))

    # #unpack and format values from nodes
    # prev_input = torch.tensor([node.pred_vocab_indxs[-1] for node in prev_input_nodes]).unsqueeze(dim=1) #dim (batch, 1)
    # hidden_tensor = torch.stack([node.hidden_tens for node in prev_input_nodes]).permute(1,0,2) # dim (layers, batch, hidden dim)
    # cell_tensor = torch.stack([node.cell_tens for node in prev_input_nodes]).permute(1,0,2) if prev_input_nodes[0].cell_tens!=None else None

    #check if any are eof token


    # print('this is prev input:')
    # display(prev_input)
    # print('prev input shape: {}'.format(prev_input.shape))


    #adjust input
    input = input.unsqueeze(1)

    print('\nthis is input')
    display(input)
    print('input shape: {}'.format(input.shape))


    print('\nthis is hidden_tensor')
    display(hidden_tens)
    print('hidden_tensor shape: {}'.format(hidden_tens.shape))

    # print('\nthis is cell_tensor')
    # display(cell_tensor)
    # print('cell_tensor shape: {}'.format(cell_tensor.shape))


    #generate predicted word & others
    hidden_tensor, cell_tensor, pred_logit = self.forward(input, hidden_tens, cell_tens, encoder_flayer)

    #convert logits to prob
    pred_prob = F.softmax(pred_logit, dim=1) #should be correct

    #turn sos and pad tokens to not be possible predictions
    pred_prob[:, sos_token] = -np.inf
    pred_prob[:, pad_token] = -np.inf

    return hidden_tensor, cell_tensor, pred_prob



In [24]:
def print_beam_nodes(node, order_count):
  print('\nNode {}:'.format(order_count))
  # print('\tParent Node:')
  # display(node.parent_node)
  print('\tPred Prob List: {}'.format(node.pred_probs))
  print('\tPred Vocab Index: {}'.format(node.pred_vocab_indxs))
  print('\tHidden Tensor:')
  display(node.hidden_tens)
  print('shape: {}'.format(node.hidden_tens.shape))



In [25]:
# def avg_log_prob(prob_list):
#   norm_score = np.sum(np.log(prob_list))/len(prob_list)
#   return norm_score

In [49]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder_args, decoder_args, sos_token=1, eos_token=2, pad_token=0):#, greedy=True, beam_width=2):
    super(Seq2Seq, self).__init__()

    #model structure
    self.encoder = EncoderNN(*encoder_args)
    self.decoder = DecoderNN(*decoder_args)

    # #set other model params to be ref later
    # self.encoder.node_type = encoder_args[0]
    # self.decoder.node_type = decoder_args[0]

    #set other model params to be ref later
    # self.max_length = max_length
    self.sos_token = sos_token
    self.eos_token = eos_token
    self.pad_token = pad_token

    # #search params
    # self.greedy = greedy
    # self.beam_width = beam_width

    # Linear layer to project encoder hidden dim to decoder hidden dim
    enc_hid = encoder_args[2]
    dec_hid = decoder_args[2]
    self.hidden_transform = nn.Linear(enc_hid, dec_hid)
    self.cell_transform = nn.Linear(enc_hid, dec_hid)

  #run encoder and initialize decoder with hidden states
  def run_enc_init_dec(self, inputs):
    #put on correct device
    device = next(self.parameters()).device
    inputs = inputs.to(device)

    #encoder
    node_class = RNN_Node if self.encoder.node_type == "rnn" else LSTM_Node
    (enc_hidden_node, enc_cell_node), enc_final_hidden_layer = self.encoder.forward(inputs)

    #initialize hidden and cell tensor
    dec_hidden_tensor, dec_cell_tensor = init_hidden(self.decoder, inputs.shape[0])

    #linear transform encoder return values - in case hidden sizes don't match
    dec_hidden_tensor[0] = self.hidden_transform(enc_hidden_node)
    if (dec_cell_tensor!=None) and (enc_cell_node!=None):
      dec_cell_tensor[0] = self.cell_transform(enc_cell_node)

    return dec_hidden_tensor, dec_cell_tensor, enc_final_hidden_layer


  #for the full system
  def forward(self, inputs, outputs):
    #put on correct device
    device = next(self.parameters()).device
#    inputs = inputs.to(device)
    outputs = outputs.to(device)

    # #encoder
    # node_class = RNN_Node if self.encoder.node_type == "rnn" else LSTM_Node
    # (enc_hidden_node, enc_cell_node), enc_final_hidden_layer = self.encoder.forward(inputs)

    # #initialize hidden and cell tensor
    # dec_hidden_tensor, dec_cell_tensor = init_hidden(self.decoder, inputs.shape[0])

    # #linear transform encoder return values - in case hidden sizes don't match
    # dec_hidden_tensor[0] = self.hidden_transform(enc_hidden_node)
    # if (dec_cell_tensor!=None) and (enc_cell_node!=None):
    #   dec_cell_tensor[0] = self.cell_transform(enc_cell_node)

    #run encoder and initialize decoder with hidden states
    dec_hidden_tensor, dec_cell_tensor, enc_final_hidden_layer = self.run_enc_init_dec(inputs)

    #decoder
    pred_out_tensor = torch.zeros(outputs.size(0), len(outputs[0]), self.decoder.vocab_size, device=device) #before was device = outputs.device

    for word_pos in range(0,len(outputs[0])):
      # print('\nWORD POSITION: {}'.format(word_pos))
      #get particular word position for entire batch and reshape to have 2d array
      word_batch = outputs[:,word_pos].unsqueeze(1)

      #decoder forward pass
      dec_hidden_tensor, dec_cell_tensor, pred_out = self.decoder.forward(word_batch, dec_hidden_tensor, dec_cell_tensor, enc_final_hidden_layer)

      #add value to prediction tensor
      pred_out_tensor[:, word_pos, :] = pred_out


    return pred_out_tensor #must make sure has shape (batch size, seq length, vocab size)

  #for greedy search, beam_width = 1. ow. beam search
  def pred(self, inputs, beam_width=1, max_length=30):

    #run encoder and initialize decoder with hidden states
    dec_hidden_tensor, dec_cell_tensor, enc_flayer = self.run_enc_init_dec(inputs)

    #dimensions
    batch_size = inputs.size(0)
    vocab_size = self.decoder.vocab_size


    #generate root nodes
    pred_probs = torch.tensor([[] for _ in range(batch_size)])

    last_pred_vocab = torch.tensor([self.sos_token for _ in range(batch_size)])
    running_pred_vocab = torch.tensor([[self.sos_token] for _ in range(batch_size)])

    hidden_tens = [dec_hidden_tensor[:,b,:] for b in range(batch_size)]
    hidden_tens = torch.stack(hidden_tens, dim=1)
    cell_tens = [dec_cell_tensor[:,b,:] for b in range(batch_size)] if dec_cell_tensor is not None else None
    cell_tens = torch.stack(cell_tens, dim=1) if cell_tens is not None else None


    print('\nIN PREDICTION')
    print('pred probs:')
    display(pred_probs)
    print('pred probs shape: {}'.format(pred_probs.shape))

    print('\npred vocab indxs:')
    display(running_pred_vocab)
    print('pred vocab indxs shape: {}'.format(running_pred_vocab.shape))

    print('\nhidden tensors')
    display(hidden_tens)
    print('hidden tensors shape: {}'.format(hidden_tens.shape))


    it = 0
    while(it<max_length):
      print('\nITERATION {}'.format(it))

      #get indexes that are eos and not eos
      eos_indxs = torch.nonzero(last_pred_vocab == self.eos_token).squeeze()
      non_eos_indx = torch.nonzero(last_pred_vocab != self.eos_token).squeeze()

      #if all sentences are completed
      if(non_eos_indx.numel() == 0):
        break

      #grab non eos items
      last_vocab_neos = last_pred_vocab[non_eos_indx]
      hidden_tens_neos = hidden_tens[:,non_eos_indx,:]
      cell_tens_neos = cell_tens[:,non_eos_indx,:] if cell_tens is not None else None

      #generate predictions based on this
      hidden_new_neos, cell_new_neos, pred_prob_new_neos = self.decoder.pred(last_vocab_neos, hidden_tens_neos, cell_tens_neos, sos_token=self.sos_token, pad_token=self.pad_token)


      #fill in missing
      hidden_tens[:,non_eos_indx,:] = hidden_new_neos
      if(cell_tens is not None):
        cell_tens[:,non_eos_indx,:] = cell_new_neos

      #create new array in full size
      tot_vals = len(last_pred_vocab)
      new_shape = (tot_vals, pred_prob_new_neos.size(1))
      pred_prob_full = torch.full(new_shape, -np.inf)
      pred_prob_full[non_eos_indx] = pred_prob_new_neos[non_eos_indx]
      pred_prob_full = pred_prob_full.reshape((batch_size,-1, pred_prob_full.size(1)))  # DOUBLE CHECK

      print('\npred prob full')
      display(pred_prob_full)
      print('pred prob full shape: {}'.format(pred_prob_full.shape))

      #flatten on appropriate axis - original should be shape [[[vocab 1][vocab 2]],[[vocab 1][vocab 2]]]
      pred_prob_full = pred_prob_full.squeeze(1)

      print('\nnew pred prob full')
      display(pred_prob_full)
      print('pred prob full shape: {}'.format(pred_prob_full.shape))

      #find top k vals
      topk_vals, topk_indx = torch.topk(pred_prob_full, beam_width, dim=1)

      print('\nTOP K Indx')
      display(topk_indx)
      print('shape: {}'.format(topk_indx.shape))

      #find out which parent it belongs to
      pred_parent_indx = (topk_indx // vocab_size).flatten()

      print('\npredicted parents')
      display(pred_parent_indx)
      print('shape: {}'.format(pred_parent_indx.shape))

      #adjust parent indx to find out what order out of orginal total it is in
      elem_per_batch = tot_vals//batch_size
      batch_adj = (torch.arange(len(pred_parent_indx))//elem_per_batch)*elem_per_batch
      pred_parent_indx_cont = pred_parent_indx + batch_adj

      #find out true vocab value
      pred_vocab_indx = (topk_indx % vocab_size).flatten()


      #### update values ####
      #get new possible running vals
      new_running_pred_vocab = torch.empty((batch_size, 1)) ## DOUBLE CHECK
      #fill in eos values
      running_pred_vocab = running_pred_vocab.reshape((batch_size, elem_per_batch, -1))
      new_running_pred_vocab = [
        running_pred_vocab[batch_idx][(eos_indxs // elem_per_batch == batch_idx).nonzero(as_tuple=True)[0]]
        for batch_idx in range(batch_size)
      ]



      #create list of dictionaries

      # batch_dic = {b:{for bw in beam_width} for b in batch_size}



      # #pred_prob_new_neos # shape should be [[vocab size][vocab size] x non eos vals]
      # pred_probs_new =

      # def fill_missing_values(eos_indx):


      return
      it+=1
      # if(it==2):
      #   break

    #create root nodes
    # nodes = np.array([[BeamNode(
    #     pred_vocab_indxs=[self.sos_token],
    #     hidden_tens=dec_hidden_tensor[:,b,:],
    #     cell_tens=dec_cell_tensor[:b,:] if dec_cell_tensor is not None else None
    # )] for b in batch_size])

    # ### TESTING ###
    # nodes = np.array([[
    #     BeamNode(
    #       # pred_probs=[-np.inf],
    #       pred_vocab_indxs=[1],
    #       hidden_tens=dec_hidden_tensor[:,b,:],
    #       cell_tens=dec_cell_tensor[:b,:] if dec_cell_tensor is not None else None),
    #     BeamNode(
    #       # pred_probs=[-np.inf],
    #       pred_vocab_indxs=[2],
    #       hidden_tens=dec_hidden_tensor[:,b,:],
    #       cell_tens=dec_cell_tensor[:b,:] if dec_cell_tensor is not None else None),
    #     BeamNode(
    #       # pred_probs=[-np.inf],
    #       pred_vocab_indxs=[3],
    #       hidden_tens=dec_hidden_tensor[:,b,:],
    #       cell_tens=dec_cell_tensor[:b,:] if dec_cell_tensor is not None else None),
    #     BeamNode(
    #       # pred_probs=[-np.inf],
    #       pred_vocab_indxs=[4],
    #       hidden_tens=dec_hidden_tensor[:,b,:],
    #       cell_tens=dec_cell_tensor[:b,:] if dec_cell_tensor is not None else None)
    #     ] for b in range(batch_size)])



    # print('\nTHESE ARE THE INPUT NODES')
    # display(nodes)
    # print('input node shape: {}'.format(nodes.shape))

    # #### just to see - testing ####
    # count = 0
    # for idx, x in np.ndenumerate(nodes):
    #   print_beam_nodes(x, count)
    #   count += 1

    # #create mask
    # size = (inputs.size(0), beam_width)
    # mask_tens = torch.ones(size, dtype=torch.bool)

    # print('\nThis is currently the mask:')
    # display(mask_tens)
    # print('shape: {}'.format(mask_tens.shape))


    # print('\n\nBEAM WIDTH: {}'.format(beam_width))
    # it = 0
    # while(mask_tens.any() and it<max_length):
      # #unstack nodes
      # input_nodes = nodes.flatten().tolist()

      # #seperate nodes that have eof_flag=1 and eof_flag=0 -- maybe replace ones with eof=1 as None


      # # #turn to list
      # # prev_input_nodes = prev_input_nodes.tolist()

      # #unpack and format values from nodes
      # prev_input = torch.tensor([node.pred_vocab_indxs[-1] for node in input_nodes]).unsqueeze(dim=1) #dim (batch, 1)
      # hidden_tensor = torch.stack([node.hidden_tens for node in input_nodes]).permute(1,0,2) # dim (layers, batch, hidden dim)
      # cell_tensor = torch.stack([node.cell_tens for node in input_nodes]).permute(1,0,2) if input_nodes[0].cell_tens!=None else None

      # #check if any prev inputs are eos
      # eos_indx = torch.nonzero(torch.eq(prev_input, self.eos_token)).squeeze().tolist()
      # non_eos_indxs = torch.nonzero(torch.ne(prev_input, self.eos_token)).squeeze().tolist()

      # #if there are no no_eos_indxs break
      # if (non_eos_indxs == []):
      #   break





      # #generate predictions on eos_flag=0
      # hidden, cell, pred_prob = self.decoder.pred(input_nodes, sos_token=self.sos_token, pad_token=self.pad_token)

      #

      # #unstack nodes to run alomost as
      # input_nodes = nodes.flatten()

      # #generate predictions
      # hidden, cell, pred_prob = self.decoder.pred(input_nodes, sos_token=self.sos_token, pad_token=self.pad_token)

      # #group pred prob by batch
      # new_dim = nodes.shape[1] * self.decoder.vocab_size
      # pred_prob = pred_prob.reshape(batch_size, new_dim)
      # print('PRED RESHAPE')
      # display(pred_prob)
      # print('shape: {}'.format(pred_prob.shape))

      # #choose amt to keep based on beam width
      # topk_vals, topk_indx = torch.topk(pred_prob, beam_width, dim=1)

      # print('\nTOP K VALUES')
      # display(topk_vals)

      # print('\nTOP K INDICES')
      # display(topk_indx)
      # #flatten for consistancy
      # topk_vals = topk_vals.flatten()

      # #find out actual value in vocabulary
      # pred_vocab_indx = (topk_indx % self.decoder.vocab_size).flatten()
      # print('\nPRED VOCAB INDX')
      # display(pred_vocab_indx)
      # print('shape: {}'.format(pred_vocab_indx.shape))


      # ##find out parent indx value
      # parent_indx = topk_indx // self.decoder.vocab_size


      # ###########just temporarily change it
      # parent_indx = torch.tensor([[1, 1, 2 ],
      #               [1, 1, 2 ],
      #               [2, 2, 1]])

      # #adjust for batch
      # row_add = (torch.arange(parent_indx.size(0)) * nodes.shape[1]).unsqueeze(1)
      # parent_indx = (parent_indx + row_add).flatten()

      # print('\nParent indx')
      # display(parent_indx)
      # print('shape: {}'.format(parent_indx.shape))

      # #take hidden and cell states accordingly
      # hidden_vals = hidden[:, parent_indx,:]
      # print('\nHIDDEN VALS')
      # display(hidden_vals)
      # print('shape: {}'.format(hidden_vals.shape))

      # cell_vals = cell[:, parent_indx,:] if cell is not None else None

      # #get the actual parent nodes
      # parent_nodes = input_nodes[parent_indx]

      # #create new nodes
      # total_node_count = len(topk_vals)
      # nodes = np.array([
      #     BeamNode(
      #         parent_node = parent_nodes[i],

      #         pred_prob = topk_vals[i],
      #         pred_vocab_indx = pred_vocab_indx[i],

      #         hidden_tens = hidden_vals[:,i,:],
      #         cell_tens = cell_vals[:,i,:] if cell_vals!=None else None #MUST DOUBLE CHECK
      #         )
      # for i in range(total_node_count) ])


      # print('\n\n\nNEW NODES:')
      # display(nodes)
      # print('shape: {}'.format(nodes.shape))

      # count = 0
      # _ = [print_beam_nodes(nodes[i],i) for i in range(0,len(nodes))]

      # #reshape accordingly
      # nodes = nodes.reshape(batch_size, beam_width)

      # print('\nNew Nodes')
      # display(nodes)
      # print('shape: {}'.format(nodes.shape))







    #       #relationship
    # self.parent_node = parent_node

    # #values
    # self.pred_prob = pred_prob
    # self.pred_vocab_indx = pred_vocab_indx

    # #recurrent states
    # self.hidden_tens = hidden_tens
    # self.cell_tens = cell_tens




      # print('\nCELL')
      # display(cell)
      # print('shape: {}'.format(cell.shape))


      #create new nodes with information

      # print('\nHIDDEN')
      # display(hidden)
      # print('shape: {}'.format(hidden.shape))

      # print('\nCELL')
      # display(cell)
      # print('shape: {}'.format(cell.shape))

      # break


      # it+=1





    return


In [ ]:
data_dict["English Vocabulary"]

In [ ]:
#have to seperate by word
output[:,0].reshape(len(output),1)
# len(output)

In [ ]:
prev_input = torch.tensor([[1], [1], [1]])
prev_input

In [ ]:
input

In [50]:

#encoder param
enc_node_type = "rnn"
enc_n_layers = 2
enc_hidden_dim = 4
enc_embed_dim = 7
eng_vocab_size = len(data_dict["English Vocabulary"])

#decoder_param
dec_node_type = "rnn"
dec_n_layers = 5
dec_hidden_dim = 6
dec_embed_dim = 8
it_vocab_size = len(data_dict["Italian Vocabulary"])

#seq2seq param
# max_length = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
greedy=True
beam_width=2
sos_token=1
eos_token=2
pad_token=0

# (self, node_type, n_layers, hidden_dim, embed_dim, vocab_size)
# (self, node_type, n_layers, hidden_dim, embed_dim, vocab_size, attention=False, max_length=30)

print('English vocab size: {}'.format(eng_vocab_size))
print('Italian vocab size: {}'.format(it_vocab_size))

#create seq2seq instance
encoder_args = (enc_node_type, enc_n_layers, enc_hidden_dim, enc_embed_dim, eng_vocab_size)
decoder_args = (dec_node_type, dec_n_layers, dec_hidden_dim, dec_embed_dim, it_vocab_size)
seq2seq = Seq2Seq(encoder_args, decoder_args).to(device)#, greedy=greedy, beam_width=beam_width).to(device)

#testing out decoder prediction
pred_out = seq2seq.pred(input, beam_width=2)
pred_out

# temp = seq2seq.forward(input, output)

# prev_input = torch.tensor([[1], [1], [1]])
# #itialize tens based on node type
# batch_size = 3
# init_hidden_tensor = torch.zeros(dec_n_layers, batch_size, dec_hidden_dim).to(device)
# init_cell_tensor = None
# #create decoder instance
# decoder = DecoderNN(*decoder_args)
# hidden_tensor, cell_tensor, pred_word_indx = decoder.pred(prev_input, init_hidden_tensor, init_cell_tensor)
# pred_word_indx

#init_hidden_tensor, init_cell_tensor = init_hidden(batch_size=3)


# #################################################

# temp = seq2seq.forward(input, output)
# print('this is seq2seq forward output')
# display(temp)
# print("output shape: {}".format(temp.shape))
# print("output type: {}".format(type(temp)))
# #forward step

# # #encoder instance
# encoder = EncoderNN(*encoder_args)
# enc_out, final_hidden_layer = encoder.forward(input)
# enc_hidden_node = enc_out[0]

# print('\n\n\nafter encoder test step')



# #decoder instance
# decoder = DecoderNN(*decoder_args)
# #initialize hidden and cell list
# dec_hidden_list, dec_cell_list = init_hidden(decoder, output.shape[0])
# dec_hidden_list[0] = enc_hidden_node

# pred_out = decoder.forward(output[:,0].reshape(len(output), 1), dec_hidden_list, None, final_hidden_layer)
# print('\nafter decoder test step')

# #create encoder instance
# encoder = EncoderNN(node_type, n_layers, hidden_dim, embed_dim, eng_vocab_size)

# #forward step
# hidden_list, final_hidden_layer = encoder.forward(input)

English vocab size: 11
Italian vocab size: 10

IN PREDICTION
pred probs:


tensor([], size=(3, 0))

pred probs shape: torch.Size([3, 0])

pred vocab indxs:


tensor([[1],
        [1],
        [1]])

pred vocab indxs shape: torch.Size([3, 1])

hidden tensors


...

pred prob full shape: torch.Size([3, 10])

TOP K Indx


tensor([[9, 8],
        [9, 8],
        [9, 8]])

shape: torch.Size([3, 2])

predicted parents


tensor([0, 0, 0, 0, 0, 0])

shape: torch.Size([6])


In [ ]:
print('true output')
display(output)
print("true output shape: {}".format(output.shape))

Notes:

- before running - write in comments how each dimension is looking like after each computation - helpful for debugging

- incorporate batches!

- transformers

- beam search

- bidirectional

- different attention methods

-- evaluate with Bleu scores


SHOULD ADD
- dependency parsing, pretraining, and fine-tuning

Run time notes:

- embedding size: Common values for the embedding dimension in NLP tasks range from 50 to 300, but larger values like 512 or 1024 can be used for more complex tasks or larger datasets.